In [1]:
%load_ext autoreload
from __future__ import print_function, division

In [2]:
%autoreload

import copy, math, os, pickle, time, pandas as pd, numpy as np, scipy.stats as ss

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import torch, torch.utils.data as utils, torch.nn as nn, torch.nn.functional as F, torch.optim as optim
from torch.autograd import Variable
from torch.nn.parameter import Parameter


#from mmd_grud_utils import *

In [3]:
DATA_FILEPATH     = '../data/grouped/all_hourly_data.h5'
RAW_DATA_FILEPATH = '../data/data/all_hourly_data.h5'
GAP_TIME          = 6  # In hours
WINDOW_SIZE       = 24 # In hours
SEED              = 1
ID_COLS           = ['subject_id', 'hadm_id', 'icustay_id']
GPU               = '2'

os.environ['CUDA_VISIBLE_DEVICES'] = GPU
np.random.seed(SEED)
torch.manual_seed(SEED)

In [4]:
class DictDist():
    def __init__(self, dict_of_rvs): self.dict_of_rvs = dict_of_rvs
    def rvs(self, n):
        a = {k: v.rvs(n) for k, v in self.dict_of_rvs.items()}
        out = []
        for i in range(n): out.append({k: vs[i] for k, vs in a.items()})
        return out
    
class Choice():
    def __init__(self, options): self.options = options
    def rvs(self, n): return [self.options[i] for i in ss.randint(0, len(self.options)).rvs(n)]

In [5]:
%%time
data_full_lvl2 = pd.read_hdf(DATA_FILEPATH, 'vitals_labs')
#data_full_raw  = pd.read_hdf(RAW_DATA_FILEPATH, 'vitals_labs') 
statics        = pd.read_hdf(DATA_FILEPATH, 'patients')

CPU times: user 1min 37s, sys: 6min 1s, total: 7min 39s
Wall time: 11min 39s


In [6]:

data_full_raw  = pd.read_hdf(RAW_DATA_FILEPATH, 'vitals_labs') 


In [7]:
data_full_lvl2.head()

LEVEL2                                 alanine aminotransferase             \
Aggregation Function                                      count  mean  std   
subject_id hadm_id icustay_id hours_in                                       
3          145834  211552     0                             2.0  25.0  0.0   
                              1                             0.0   NaN  NaN   
                              2                             0.0   NaN  NaN   
                              3                             0.0   NaN  NaN   
                              4                             0.0   NaN  NaN   

LEVEL2                                 albumin           albumin ascites       \
Aggregation Function                     count mean  std           count mean   
subject_id hadm_id icustay_id hours_in                                          
3          145834  211552     0            2.0  1.8  0.0             0.0  NaN   
                              1            0.0  NaN  NaN             0.0  NaN   
                              2            0.0  NaN  NaN             0.0  NaN   
                              3            0.0  NaN  NaN             0.0  NaN   
                              4            0.0  NaN  NaN             0.0  NaN   

LEVEL2                                     albumin pleural  ...  \
Aggregation Function                   std           count  ...   
subject_id hadm_id icustay_id hours_in                      ...   
3          145834  211552     0        NaN             0.0  ...   
                              1        NaN             0.0  ...   
                              2        NaN             0.0  ...   
                              3        NaN             0.0  ...   
                              4        NaN             0.0  ...   

LEVEL2                                 white blood cell count  \
Aggregation Function                                      std   
subject_id hadm_id icustay_id hours_in                          
3          145834  211552     0                      4.012837   
                              1                           NaN   
                              2                           NaN   
                              3                           NaN   
                              4                           NaN   

LEVEL2                                 white blood cell count urine           \
Aggregation Function                                          count mean std   
subject_id hadm_id icustay_id hours_in                                         
3          145834  211552     0                                 0.0  NaN NaN   
                              1                                 0.0  NaN NaN   
                              2                                 0.0  NaN NaN   
                              3                                 0.0  NaN NaN   
                              4                                 0.0  NaN NaN   

LEVEL2                                    ph                 ph urine           
Aggregation Function                   count  mean       std    count mean std  
subject_id hadm_id icustay_id hours_in                                          
3          145834  211552     0          9.0  7.40  0.147733      1.0  5.0 NaN  
                              1          0.0   NaN       NaN      0.0  NaN NaN  
                              2          3.0  7.26  0.000000      0.0  NaN NaN  
                              3          0.0   NaN       NaN      0.0  NaN NaN  
                              4          0.0   NaN       NaN      0.0  NaN NaN  

[5 rows x 312 columns]

In [8]:
#data_full_raw.head(20)

In [9]:
statics.head()

,,,gender,ethnicity,age,insurance,admittime,diagnosis_at_admission,dischtime,discharge_location,fullcode_first,dnr_first,...,outtime,los_icu,admission_type,first_careunit,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,readmission_30,max_hours
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,,,,,
3,145834,211552,M,WHITE,76.526792,Medicare,2101-10-20 19:08:00,HYPOTENSION,2101-10-31 13:58:00,SNF,1.0,0.0,...,2101-10-26 20:43:09,6.064560,EMERGENCY,MICU,0,0,0,1,0,145
4,185777,294638,F,WHITE,47.845047,Private,2191-03-16 00:28:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",2191-03-23 18:41:00,HOME WITH HOME IV PROVIDR,1.0,0.0,...,2191-03-17 16:46:31,1.678472,EMERGENCY,MICU,0,0,0,1,0,40
6,107064,228232,F,WHITE,65.942297,Medicare,2175-05-30 07:15:00,CHRONIC RENAL FAILURE/SDA,2175-06-15 16:00:00,HOME HEALTH CARE,1.0,0.0,...,2175-06-03 13:39:54,3.672917,ELECTIVE,SICU,0,0,0,1,0,88
9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41.790228,Medicaid,2149-11-09 13:06:00,HEMORRHAGIC CVA,2149-11-14 10:15:00,DEAD/EXPIRED,1.0,0.0,...,2149-11-14 20:52:14,5.323056,EMERGENCY,MICU,1,1,1,1,0,127
11,194540,229441,F,WHITE,50.148295,Private,2178-04-16 06:18:00,BRAIN MASS,2178-05-11 19:00:00,HOME HEALTH CARE,1.0,0.0,...,2178-04-17 20:21:05,1.584410,EMERGENCY,SICU,0,0,0,1,0,38


In [10]:
def simple_imputer(df):
    idx = pd.IndexSlice
    df = df.copy()
    if len(df.columns.names) > 2: df.columns = df.columns.droplevel(('label', 'LEVEL1', 'LEVEL2'))
    
    df_out = df.loc[:, idx[:, ['mean', 'count']]]
    icustay_means = df_out.loc[:, idx[:, 'mean']].groupby(ID_COLS).mean()
    
    df_out.loc[:,idx[:,'mean']] = df_out.loc[:,idx[:,'mean']].groupby(ID_COLS).fillna(
        method='ffill'
    ).groupby(ID_COLS).fillna(icustay_means).fillna(0)
    
    df_out.loc[:, idx[:, 'count']] = (df.loc[:, idx[:, 'count']] > 0).astype(float)
    df_out.rename(columns={'count': 'mask'}, level='Aggregation Function', inplace=True)
    
    is_absent = (1 - df_out.loc[:, idx[:, 'mask']])
    hours_of_absence = is_absent.cumsum()
    time_since_measured = hours_of_absence - hours_of_absence[is_absent==0].fillna(method='ffill')
    time_since_measured.rename(columns={'mask': 'time_since_measured'}, level='Aggregation Function', inplace=True)

    df_out = pd.concat((df_out, time_since_measured), axis=1)
    df_out.loc[:, idx[:, 'time_since_measured']] = df_out.loc[:, idx[:, 'time_since_measured']].fillna(100)
    
    df_out.sort_index(axis=1, inplace=True)
    return df_out

In [11]:
Ys = statics[statics.max_hours > WINDOW_SIZE + GAP_TIME][['mort_hosp', 'mort_icu', 'los_icu']]
Ys['los_3'] = Ys['los_icu'] > 3
Ys['los_7'] = Ys['los_icu'] > 7
Ys.drop(columns=['los_icu'], inplace=True)
Ys.astype(float)

lvl2, raw = [df[
    (df.index.get_level_values('icustay_id').isin(set(Ys.index.get_level_values('icustay_id')))) &
    (df.index.get_level_values('hours_in') < WINDOW_SIZE)
] for df in (data_full_lvl2, data_full_raw)]

#raw.columns = raw.columns.droplevel(level=['label', 'LEVEL1', 'LEVEL2'])

train_frac, dev_frac, test_frac = 0.7, 0.1, 0.2
lvl2_subj_idx, raw_subj_idx, Ys_subj_idx = [df.index.get_level_values('subject_id') for df in (lvl2, raw, Ys)]
lvl2_subjects = set(lvl2_subj_idx)
assert lvl2_subjects == set(Ys_subj_idx), "Subject ID pools differ!"
#assert lvl2_subjects == set(raw_subj_idx), "Subject ID pools differ!"

np.random.seed(SEED)
subjects, N = np.random.permutation(list(lvl2_subjects)), len(lvl2_subjects)
N_train, N_dev, N_test = int(train_frac * N), int(dev_frac * N), int(test_frac * N)
train_subj = subjects[:N_train]
dev_subj   = subjects[N_train:N_train + N_dev]
test_subj  = subjects[N_train+N_dev:]

[(lvl2_train, lvl2_dev, lvl2_test), (raw_train, raw_dev, raw_test), (Ys_train, Ys_dev, Ys_test)] = [
    [df[df.index.get_level_values('subject_id').isin(s)] for s in (train_subj, dev_subj, test_subj)] \
    for df in (lvl2, raw, Ys)
]

idx = pd.IndexSlice
lvl2_means, lvl2_stds = lvl2_train.loc[:, idx[:,'mean']].mean(axis=0), lvl2_train.loc[:, idx[:,'mean']].std(axis=0)
#raw_means, raw_stds = raw_train.loc[:, idx[:,'mean']].mean(axis=0), raw_train.loc[:, idx[:,'mean']].std(axis=0)

lvl2_train.loc[:, idx[:,'mean']] = (lvl2_train.loc[:, idx[:,'mean']] - lvl2_means)/lvl2_stds
lvl2_dev.loc[:, idx[:,'mean']] = (lvl2_dev.loc[:, idx[:,'mean']] - lvl2_means)/lvl2_stds
lvl2_test.loc[:, idx[:,'mean']] = (lvl2_test.loc[:, idx[:,'mean']] - lvl2_means)/lvl2_stds

#raw_train.loc[:, idx[:,'mean']] = (raw_train.loc[:, idx[:,'mean']] - raw_means)/raw_stds
#raw_dev.loc[:, idx[:,'mean']] = (raw_dev.loc[:, idx[:,'mean']] - raw_means)/raw_stds
#raw_test.loc[:, idx[:,'mean']] = (raw_test.loc[:, idx[:,'mean']] - raw_means)/raw_stds

/Users/mengxuan/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1884: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


In [12]:

#raw_train, raw_dev, raw_test, lvl2_train, lvl2_dev, lvl2_test = [
#    simple_imputer(df) for df in (raw_train, raw_dev, raw_test, lvl2_train, lvl2_dev, lvl2_test)
#]
#raw_flat_train, raw_flat_dev, raw_flat_test, lvl2_flat_train, lvl2_flat_dev, lvl2_flat_test = [
#    df.pivot_table(index=['subject_id', 'hadm_id', 'icustay_id'], columns=['hours_in']) for df in (
#        raw_train, raw_dev, raw_test, lvl2_train, lvl2_dev, lvl2_test
#    )
#]

#for df in lvl2_train, lvl2_dev, lvl2_test, raw_train, raw_dev, raw_test: assert not df.isnull().any().any()

In [13]:
lvl2_train, lvl2_dev, lvl2_test = [
    simple_imputer(df) for df in (lvl2_train, lvl2_dev, lvl2_test)
]
lvl2_flat_train, lvl2_flat_dev, lvl2_flat_test = [
    df.pivot_table(index=['subject_id', 'hadm_id', 'icustay_id'], columns=['hours_in']) for df in (
        lvl2_train, lvl2_dev, lvl2_test
    )
]

for df in lvl2_train, lvl2_dev, lvl2_test: assert not df.isnull().any().any()


/Users/mengxuan/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [14]:
Ys = statics[statics.max_hours > WINDOW_SIZE + GAP_TIME][['mort_hosp', 'mort_icu', 'los_icu']]
Ys['los_3'] = Ys['los_icu'] > 3
Ys['los_7'] = Ys['los_icu'] > 7
Ys.drop(columns=['los_icu'], inplace=True)
Ys.astype(float)
[(Ys_train, Ys_dev, Ys_test)] = [
    [df[df.index.get_level_values('subject_id').isin(s)] for s in (train_subj, dev_subj, test_subj)] \
    for df in (Ys,)
]

### Task Prediction

#### Hyperparams

In [15]:
N = 10

GRU_D_dist = DictDist({
    'cell_size': ss.randint(50, 75),
    'hidden_size': ss.randint(65, 95), 
    'learning_rate': ss.uniform(2e-3, 1e-1),
    'num_epochs': ss.randint(15, 150),
    'patience': ss.randint(3, 7),
    'batch_size': ss.randint(35, 65),
    'early_stop_frac': ss.uniform(0.05, 0.1),
    'seed': ss.randint(1, 10000),
})
np.random.seed(SEED)
GRU_D_hyperparams_list = GRU_D_dist.rvs(N)

#with open('../scratch/mmd/baselines_gru-d.pkl', mode='rb') as f: results = pickle.load(f)

### GRU-D Model

In [16]:
import copy, math, os, pickle, time, pandas as pd, numpy as np, scipy.stats as ss

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import torch, torch.utils.data as utils, torch.nn as nn, torch.nn.functional as F, torch.optim as optim
from torch.autograd import Variable
from torch.nn.parameter import Parameter

def to_3D_tensor(df):
    idx = pd.IndexSlice
    return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))
def prepare_dataloader(df, Ys, batch_size, shuffle=True):
    """
    dfs = (df_train, df_dev, df_test).
    df_* = (subject, hadm, icustay, hours_in) X (level2, agg fn \ni {mask, mean, time})
    Ys_series = (subject, hadm, icustay) => label.
    """
    X     = torch.from_numpy(to_3D_tensor(df).astype(np.float32))
    #print(X[:5])
    label = torch.from_numpy(Ys.values.astype(np.int64))
    dataset = utils.TensorDataset(X, label)
    
    return utils.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last = True)

class FilterLinear(nn.Module):
    def __init__(self, in_features, out_features, filter_square_matrix, bias=True):
        '''
        filter_square_matrix : filter square matrix, whose each elements is 0 or 1.
        '''
        super(FilterLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        assert in_features > 1 and out_features > 1, "Passing in nonsense sizes"
        
        use_gpu = torch.cuda.is_available()
        self.filter_square_matrix = None
        if use_gpu: self.filter_square_matrix = Variable(filter_square_matrix.cuda(), requires_grad=False)
        else:       self.filter_square_matrix = Variable(filter_square_matrix, requires_grad=False)
        
        self.weight = Parameter(torch.Tensor(out_features, in_features))

        if bias: self.bias = Parameter(torch.Tensor(out_features))
        else:    self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None: self.bias.data.uniform_(-stdv, stdv)

    def forward(self, x):
        return F.linear(
            x,
            self.filter_square_matrix.mul(self.weight),
            self.bias
        )

    def __repr__(self):
        return self.__class__.__name__ + '(' \
            + 'in_features=' + str(self.in_features) \
            + ', out_features=' + str(self.out_features) \
            + ', bias=' + str(self.bias is not None) + ')'
        


In [17]:

class GRUD(nn.Module):
    def __init__(self, input_size, cell_size, hidden_size, X_mean, batch_size = 0, output_last = False):
        """
        With minor modifications from https://github.com/zhiyongc/GRU-D/

        Recurrent Neural Networks for Multivariate Times Series with Missing Values
        GRU-D: GRU exploit two representations of informative missingness patterns, i.e., masking and time interval.
        cell_size is the size of cell_state.
        
        Implemented based on the paper: 
        @article{che2018recurrent,
          title={Recurrent neural networks for multivariate time series with missing values},
          author={Che, Zhengping and Purushotham, Sanjay and Cho, Kyunghyun and Sontag, David and Liu, Yan},
          journal={Scientific reports},
          volume={8},
          number={1},
          pages={6085},
          year={2018},
          publisher={Nature Publishing Group}
        }
        
        GRU-D:
            input_size: variable dimension of each time
            hidden_size: dimension of hidden_state
            mask_size: dimension of masking vector
            X_mean: the mean of the historical input data
        """
        
        super(GRUD, self).__init__()
        
        self.hidden_size = hidden_size
        self.delta_size = input_size
        self.mask_size = input_size
        
        use_gpu = torch.cuda.is_available()
        if use_gpu:
            self.identity = torch.eye(input_size).cuda()
            self.zeros = Variable(torch.zeros(batch_size, input_size).cuda())
            self.zeros_h = Variable(torch.zeros(batch_size, self.hidden_size).cuda())
            self.X_mean = Variable(torch.Tensor(X_mean).cuda())
        else:
            self.identity = torch.eye(input_size)
            self.zeros = Variable(torch.zeros(batch_size, input_size))
            self.zeros_h = Variable(torch.zeros(batch_size, self.hidden_size))
            self.X_mean = Variable(torch.Tensor(X_mean))
        
        self.zl = nn.Linear(input_size + hidden_size + self.mask_size, hidden_size) # Wz, Uz are part of the same network. the bias is bz
        self.rl = nn.Linear(input_size + hidden_size + self.mask_size, hidden_size) # Wr, Ur are part of the same network. the bias is br
        self.hl = nn.Linear(input_size + hidden_size + self.mask_size, hidden_size) # W, U are part of the same network. the bias is b
        
        self.gamma_x_l = FilterLinear(self.delta_size, self.delta_size, self.identity)
        
        self.gamma_h_l = nn.Linear(self.delta_size, self.hidden_size) # this was wrong in available version. remember to raise the issue
        
        self.output_last = output_last
        
        self.fc = nn.Linear(self.hidden_size, 2)
        self.bn= torch.nn.BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True)
        self.drop=nn.Dropout(p=0.5, inplace=False)
        
    def step(self, x, x_last_obsv, x_mean, h, mask, delta):
        """
        Inputs:
            x: input tensor
            x_last_obsv: input tensor with forward fill applied
            x_mean: the mean of each feature
            h: the hidden state of the network
            mask: the mask of whether or not the current value is observed
            delta: the tensor indicating the number of steps since the last time a feature was observed.
            
        Returns:
            h: the updated hidden state of the network
        """
        
        batch_size = x.size()[0]
        dim_size = x.size()[1]
        
        gamma_x_l_delta = self.gamma_x_l(delta)
        delta_x = torch.exp(-torch.max(self.zeros, gamma_x_l_delta)) #exponentiated negative rectifier
        
        gamma_h_l_delta = self.gamma_h_l(delta)
        delta_h = torch.exp(-torch.max(self.zeros_h, gamma_h_l_delta)) #self.zeros became self.zeros_h to accomodate hidden size != input size
        
        x_mean = x_mean.repeat(batch_size, 1)
   
        x = mask * x + (1 - mask) * (delta_x * x_last_obsv + (1 - delta_x) * x_mean)
        x = torch.nan_to_num(x)
        
        h = delta_h * h
       
        combined = torch.cat((x, h, mask), 1)
        z = torch.sigmoid(self.zl(combined)) #sigmoid(W_z*x_t + U_z*h_{t-1} + V_z*m_t + bz)
        r = torch.sigmoid(self.rl(combined)) #sigmoid(W_r*x_t + U_r*h_{t-1} + V_r*m_t + br)
        combined_new = torch.cat((x, r*h, mask), 1)
        h_tilde = torch.tanh(self.hl(combined_new)) #tanh(W*x_t +U(r_t*h_{t-1}) + V*m_t) + b
        h = (1 - z) * h + z * h_tilde
        h = torch.nan_to_num(h)
        return h
    
    def forward(self, X, X_last_obsv, Mask, Delta):
        batch_size = X.size(0)
#         type_size = input.size(1)
        step_size = X.size(1) # num timepoints
        spatial_size = X.size(2) # num features
        
        Hidden_State = self.initHidden(batch_size)
#         X = torch.squeeze(input[:,0,:,:])
#         X_last_obsv = torch.squeeze(input[:,1,:,:])
#         Mask = torch.squeeze(input[:,2,:,:])
#         Delta = torch.squeeze(input[:,3,:,:])
        
        outputs = None
        for i in range(step_size):
            Hidden_State = self.step(
                torch.squeeze(X[:,i:i+1,:], 1),
                torch.squeeze(X_last_obsv[:,i:i+1,:], 1),
                torch.squeeze(self.X_mean[:,i:i+1,:], 1),
                Hidden_State,
                torch.squeeze(Mask[:,i:i+1,:], 1),
                torch.squeeze(Delta[:,i:i+1,:], 1),
            )
            if outputs is None:
                outputs = Hidden_State.unsqueeze(1)
            else:
                outputs = torch.cat((Hidden_State.unsqueeze(1), outputs), 1)
                
        # we want to predict a binary outcome
        #Apply 50% dropout and batch norm here
        self.drop(self.bn(self.fc(Hidden_State)))
        return self.drop(self.bn(self.fc(Hidden_State)))
                
#         if self.output_last:
#             return outputs[:,-1,:]
#         else:
#             return outputs
    
    def initHidden(self, batch_size):
        use_gpu = torch.cuda.is_available()
        if use_gpu:
            Hidden_State = Variable(torch.zeros(batch_size, self.hidden_size).cuda())
            return Hidden_State
        else:
            Hidden_State = Variable(torch.zeros(batch_size, self.hidden_size))
            return Hidden_State

        


In [18]:

def Train_Model(
    model, train_dataloader, valid_dataloader, num_epochs = 300, patience = 3, min_delta = 1e-5, learning_rate=1e-3, batch_size=None
):
    
    print('Model Structure: ', model)
    print('Start Training ... ')
    
    model
    
    if (type(model) == nn.modules.container.Sequential):
        output_last = model[-1].output_last
        print('Output type dermined by the last layer')
    else:
        output_last = model.output_last
        print('Output type dermined by the model')
        
    loss_MSE = torch.nn.MSELoss()
    loss_nll=torch.nn.NLLLoss()
    loss_CEL=torch.nn.CrossEntropyLoss()
    loss_L1 = torch.nn.L1Loss()
    
    learning_rate = 0.001
#     optimizer = torch.optim.RMSprop(model.parameters(), lr = learning_rate, alpha=0.99)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    use_gpu = False#torch.cuda.is_available()
    
    interval = 100
    losses_train = []
    losses_valid = []
    losses_epochs_train = []
    losses_epochs_valid = []
    
    cur_time = time.time()
    pre_time = time.time()
    
    # Variables for Early Stopping
    is_best_model = 0
    patient_epoch = 0
    for epoch in range(num_epochs):
        
        trained_number = 0
        
        valid_dataloader_iter = iter(valid_dataloader)
        
        losses_epoch_train = []
        losses_epoch_valid = []
        
        for X, labels in train_dataloader:
            X = X.numpy()
         
            mask        = torch.from_numpy(X[:, np.arange(0, X.shape[1], 3), :].astype(np.float32))
            measurement = torch.from_numpy(X[:, np.arange(1, X.shape[1], 3), :].astype(np.float32))
            time_       = torch.from_numpy(X[:, np.arange(2, X.shape[1], 3), :].astype(np.float32))
            
            mask = torch.transpose(mask, 1, 2)
            measurement = torch.transpose(measurement, 1, 2)
            time_ = torch.transpose(time_, 1, 2)
            measurement_last_obsv = measurement            

            assert measurement.size()[0] == batch_size, "Batch Size doesn't match! %s" % str(measurement.size())

            if use_gpu:
                convert_to_cuda=lambda x: Variable(x.cuda())
                X, X_last_obsv, Mask, Delta, labels = map(convert_to_cuda, [measurement, measurement_last_obsv, mask, time_, labels])
            else: 
#                 inputs, labels = Variable(inputs), Variable(labels)
                convert_to_tensor=lambda x: Variable(x)
                X, X_last_obsv, Mask, Delta, labels  = map(convert_to_tensor, [measurement, measurement_last_obsv, mask, time_, labels])
            
            model.zero_grad()

#             outputs = model(inputs)
            prediction=model(X, X_last_obsv, Mask, Delta)
    
#             print(torch.sum(torch.sum(torch.isnan(prediction))))
            
#             print(labels.shape)
#             print(prediction.shape)
            
            if output_last:
                loss_train = loss_CEL(torch.squeeze(prediction), torch.squeeze(labels))
            else:
                full_labels = torch.cat((inputs[:,1:,:], labels), dim = 1)
                loss_train = loss_MSE(outputs, full_labels)
                
      
            losses_train.append(loss_train.data)
            losses_epoch_train.append(loss_train.data)
            
            optimizer.zero_grad()
            
            loss_train.backward()
            
            optimizer.step()
            
             # validation 
            try: 
                X_val, labels_val = next(valid_dataloader_iter)
                X_val = X_val.numpy()
                mask_val        = torch.from_numpy(X_val[:, np.arange(0, X_val.shape[1], 3), :].astype(np.float32))
                measurement_val = torch.from_numpy(X_val[:, np.arange(1, X_val.shape[1], 3), :].astype(np.float32))
                time_val       = torch.from_numpy(X_val[:, np.arange(2, X_val.shape[1], 3), :].astype(np.float32))
            
                mask_val = torch.transpose(mask_val, 1, 2)
                measurement_val = torch.transpose(measurement_val, 1, 2)
                time_val = torch.transpose(time_val, 1, 2)
                measurement_last_obsv_val = measurement_val
            except StopIteration:
                valid_dataloader_iter = iter(valid_dataloader)
                X_val, labels_val = next(valid_dataloader_iter)
                X_val = X_val.numpy()
                mask_val        = torch.from_numpy(X_val[:, np.arange(0, X_val.shape[1], 3), :].astype(np.float32))
                measurement_val = torch.from_numpy(X_val[:, np.arange(1, X_val.shape[1], 3), :].astype(np.float32))
                time_val       = torch.from_numpy(X_val[:, np.arange(2, X_val.shape[1], 3), :].astype(np.float32))
            
                mask_val = torch.transpose(mask_val, 1, 2)
                measurement_val = torch.transpose(measurement_val, 1, 2)
                time_val = torch.transpose(time_val, 1, 2)
                measurement_last_obsv_val = measurement_val
            
            if use_gpu:
                convert_to_cuda=lambda x: Variable(x.cuda())
                X_val, X_last_obsv_val, Mask_val, Delta_val, labels_val = map(convert_to_cuda, [measurement_val, measurement_last_obsv_val, mask_val, time_val, labels_val])
            else: 
#                 inputs, labels = Variable(inputs), Variable(labels)
                convert_to_tensor=lambda x: Variable(x)
                X_val, X_last_obsv_val, Mask_val, Delta_val, labels_val = map(convert_to_tensor, [measurement_val, measurement_last_obsv_val, mask_val, time_val, labels_val])
            
                
            model.zero_grad()
            
#             outputs_val = model(inputs_val)
            prediction_val = model(X_val, X_last_obsv_val, Mask_val, Delta_val)
    
#             print(labels.shape)
#             print(prediction_val.shape)
            
            if output_last:
                loss_valid =loss_CEL(torch.squeeze(prediction_val), torch.squeeze(labels_val))
            else:
                raise NotImplementedError("Should be output last!")
                full_labels_val = torch.cat((inputs_val[:,1:,:], labels_val), dim = 1)
                loss_valid = loss_MSE(outputs_val, full_labels_val)
                
          
            losses_valid.append(loss_valid.data)
            losses_epoch_valid.append(loss_valid.data)
            
#             print(sklearn.metrics.roc_auc_score(labels_val.detach().cpu().numpy(), prediction_val.detach().cpu().numpy()[:,1]))
            
            # output
            trained_number += 1
            
        avg_losses_epoch_train = sum(losses_epoch_train).cpu().numpy() / float(len(losses_epoch_train))
        avg_losses_epoch_valid = sum(losses_epoch_valid).cpu().numpy() / float(len(losses_epoch_valid))
        losses_epochs_train.append(avg_losses_epoch_train)
        losses_epochs_valid.append(avg_losses_epoch_valid)
        
        
        # Early Stopping
        if epoch == 0:
            is_best_model = 1
            best_model = model
            min_loss_epoch_valid = 10000.0
            if avg_losses_epoch_valid < min_loss_epoch_valid:
                min_loss_epoch_valid = avg_losses_epoch_valid
        else:
            if min_loss_epoch_valid - avg_losses_epoch_valid > min_delta:
                is_best_model = 1
                best_model = model
                min_loss_epoch_valid = avg_losses_epoch_valid 
                patient_epoch = 0
            else:
                is_best_model = 0
                patient_epoch += 1
                if patient_epoch >= patience:
                    print('Early Stopped at Epoch:', epoch)
                    break
        
        # Print training parameters
        cur_time = time.time()
        print('Epoch: {}, train_loss: {}, valid_loss: {}, time: {}, best model: {}'.format( \
                    epoch, \
                    np.around(avg_losses_epoch_train, decimals=8),\
                    np.around(avg_losses_epoch_valid, decimals=8),\
                    np.around([cur_time - pre_time] , decimals=2),\
                    is_best_model) )
        pre_time = cur_time
#         if epoch==1:
#             break
                
    return best_model, [losses_train, losses_valid, losses_epochs_train, losses_epochs_valid]



In [19]:

def predict_proba(model, dataloader):
    """
    Input:
        model: GRU-D model
        test_dataloader: containing batches of measurement, measurement_last_obsv, mask, time_, labels
    Returns:
        predictions: size[num_samples, 2]
        labels: size[num_samples]
    """
    model.eval()
    use_gpu = False# torch.cuda.is_available()
    
    probabilities = []
    labels        = []
    ethnicities   = []
    genders       = []
    for X, label in dataloader:
        X = X.numpy()
        mask        = torch.from_numpy(X[:, np.arange(0, X.shape[1], 3), :].astype(np.float32))
        measurement = torch.from_numpy(X[:, np.arange(1, X.shape[1], 3), :].astype(np.float32))
        time_       = torch.from_numpy(X[:, np.arange(2, X.shape[1], 3), :].astype(np.float32))

        mask = torch.transpose(mask, 1, 2)
        measurement = torch.transpose(measurement, 1, 2)
        time_ = torch.transpose(time_, 1, 2)
        measurement_last_obsv = measurement            

        if use_gpu:
            convert_to_cuda=lambda x: Variable(x.cuda())
            X, X_last_obsv, Mask, Delta, label = map(convert_to_cuda, [measurement, measurement_last_obsv, mask, time_, label])
        else: 
#                 inputs, labels = Variable(inputs), Variable(labels)
            convert_to_tensor=lambda x: Variable(x)
            X, X_last_obsv, Mask, Delta, label  = map(convert_to_tensor, [measurement, measurement_last_obsv, mask, time_, label])

        
        prob = model(X, X_last_obsv, Mask, Delta)
        
        probabilities.append(prob.detach().cpu().data.numpy())
        labels.append(label.detach().cpu().data.numpy())

    return probabilities, labels


### GRU-D

In [20]:
%%time
model_name       = 'GRU-D'
hyperparams_list = GRU_D_hyperparams_list
RERUN            = False
results = {}
if model_name not in results: results[model_name] = {}
for t in [
    'mort_icu',
#    'los_3',
#    'mort_hosp',
#    'los_7'
]:
    if t not in results[model_name]: results[model_name][t] = {}
    for n, X_train, X_dev, X_test in (
        ('lvl2', lvl2_train, lvl2_dev, lvl2_test),
#         ('raw', raw_train, raw_dev, raw_test)
    ):
        print("Running model %s on target %s with representation %s" % (model_name, t, n))
        X_mean = np.nanmean(
            to_3D_tensor(
                X_train.loc[:, pd.IndexSlice[:, 'mean']] * 
                np.where((X_train.loc[:, pd.IndexSlice[:, 'mask']] == 1).values, 1, np.NaN)
            ),
            axis=0, keepdims=True
        ).transpose([0, 2, 1])
        base_params = {'X_mean': X_mean, 'output_last': True, 'input_size': X_mean.shape[2]}
    
        if n in results[model_name][t]:
            if not RERUN: 
                print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
                print(results[model_name][t][n])
                continue
            best_s, best_hyperparams = results[model_name][t][n][-1], results[model_name][t][n][1]
            print("Loading best hyperparams", best_hyperparams)
        else:
            best_s, best_hyperparams = -np.Inf, None
            for i, hyperparams in enumerate(hyperparams_list):
                print("On sample %d / %d (hyperparams = %s)" % (i+1, len(hyperparams_list), repr((hyperparams))))

                early_stop_frac,batch_size,seed = [hyperparams[k] for k in ('early_stop_frac','batch_size','seed')]
                batch_size = int(batch_size)

                np.random.seed(seed)
                all_train_subjects = list(
                    np.random.permutation(Ys_train.index.get_level_values('subject_id').values)
                )
                N_early_stop        = int(len(all_train_subjects) * early_stop_frac)
                train_subjects      = all_train_subjects[:-N_early_stop]
                early_stop_subjects = all_train_subjects[-N_early_stop:]
                X_train_obs         = X_train[X_train.index.get_level_values('subject_id').isin(train_subjects)]
                Ys_train_obs        = Ys_train[Ys_train.index.get_level_values('subject_id').isin(train_subjects)]

                X_train_early_stop  = X_train[X_train.index.get_level_values('subject_id').isin(early_stop_subjects)]
                Ys_train_early_stop = Ys_train[
                    Ys_train.index.get_level_values('subject_id').isin(early_stop_subjects)
                ]

                train_dataloader      = prepare_dataloader(X_train_obs, Ys_train_obs[t], batch_size=batch_size)
                early_stop_dataloader = prepare_dataloader(
                    X_train_early_stop, Ys_train_early_stop[t], batch_size=batch_size
                )
                dev_dataloader        = prepare_dataloader(X_dev, Ys_dev[t], batch_size=batch_size)
                test_dataloader       = prepare_dataloader(X_test, Ys_test[t], batch_size=batch_size)

                model_hyperparams = copy.copy(base_params)
                model_hyperparams.update(
                    {k: v for k, v in hyperparams.items() if k in ('cell_size', 'hidden_size', 'batch_size')}
                )
                
                batch_size = int(batch_size)
                model = GRUD(**model_hyperparams)

                best_model, _ = Train_Model(
                    model, train_dataloader, early_stop_dataloader,
                    **{k: v for k, v in hyperparams.items() if k in (
                        'num_epochs', 'patience', 'learning_rate', 'batch_size'
                    )}
                )

                probabilities_dev, labels_dev = predict_proba(best_model, dev_dataloader)
                probabilities_dev = np.concatenate(probabilities_dev)[:, 1]
                labels_dev        = np.concatenate(labels_dev)
                s = roc_auc_score(labels_dev, probabilities_dev)
                if s > best_s:
                    best_s, best_hyperparams = s, hyperparams
                    print("New Best Score: %.2f @ hyperparams = %s" % (100*best_s, repr((best_hyperparams))))
                
        ## Test
        try:
            np.random.seed(seed)
            hyperparams = best_hyperparams # In case I forgot a replace below
            early_stop_frac,batch_size,seed = [best_hyperparams[k] for k in ('early_stop_frac','batch_size','seed')]
            batch_size = int(batch_size)
            X_train_concat, Ys_train_concat = pd.concat((X_train, X_dev)), pd.concat((Ys_train, Ys_dev))

            all_train_subjects = list(np.random.permutation(Ys_train_concat.index.get_level_values('subject_id').values))
            N_early_stop = int(len(all_train_subjects) * early_stop_frac)
            train_subjects, early_stop_subjects = all_train_subjects[:-N_early_stop], all_train_subjects[-N_early_stop:]
            X_train_obs         = X_train_concat[X_train_concat.index.get_level_values('subject_id').isin(train_subjects)]
            Ys_train_obs        = Ys_train_concat[Ys_train_concat.index.get_level_values('subject_id').isin(train_subjects)]

            X_train_early_stop  = X_train_concat[X_train_concat.index.get_level_values('subject_id').isin(early_stop_subjects)]
            Ys_train_early_stop = Ys_train_concat[Ys_train_concat.index.get_level_values('subject_id').isin(early_stop_subjects)]

            train_dataloader      = prepare_dataloader(X_train_obs, Ys_train_obs[t], batch_size=batch_size)
            early_stop_dataloader = prepare_dataloader(X_train_early_stop, Ys_train_early_stop[t], batch_size=batch_size)
            test_dataloader       = prepare_dataloader(X_test, Ys_test[t], batch_size=batch_size)

            model_hyperparams = copy.copy(base_params)
            model_hyperparams.update(
                {k: v for k, v in best_hyperparams.items() if k in ('cell_size', 'hidden_size', 'batch_size')}
            )
            model = GRUD(**model_hyperparams)

            best_model, (losses_train, losses_early_stop, losses_epochs_train, losses_epochs_early_stop) = Train_Model(
                model, train_dataloader, early_stop_dataloader,
                **{k: v for k, v in best_hyperparams.items() if k in (
                    'num_epochs', 'patience', 'learning_rate', 'batch_size'
                )}
            )

            probabilities_test, labels_test = predict_proba(best_model, test_dataloader)

            y_score = np.concatenate(probabilities_test)[:, 1]
            y_pred  = np.argmax(probabilities_test)
            y_true  = np.concatenate(labels_test)

            auc   = roc_auc_score(y_true, y_score)
            auprc = average_precision_score(y_true, y_score)
            acc   = accuracy_score(y_true, y_pred)
            F1    = f1_score(y_true, y_pred)
            print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
            print(auc, auprc, acc, F1)

            results[model_name][t][n] = None, best_hyperparams, auc, auprc, acc, F1, best_s
            #with open('../scratch/mmd/baselines_gru-d.pkl', mode='wb') as f: pickle.dump(results, f)
        except: pass


Running model GRU-D on target mort_icu with representation lvl2


/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_32257/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))
<timed exec>:18: RuntimeWarning: Mean of empty slice


On sample 1 / 10 (hyperparams = {'cell_size': 55, 'hidden_size': 66, 'learning_rate': 0.07052195003967596, 'num_epochs': 22, 'patience': 6, 'batch_size': 60, 'early_stop_frac': 0.051828827734419186, 'seed': 9496})
Model Structure:  GRUD(
  (zl): Linear(in_features=274, out_features=66, bias=True)
  (rl): Linear(in_features=274, out_features=66, bias=True)
  (hl): Linear(in_features=274, out_features=66, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=66, bias=True)
  (fc): Linear(in_features=66, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.74269202, valid_loss: 0.73621848, time: [22.24], best model: 1
Epoch: 1, train_loss: 0.5305459, valid_loss: 0.54031748, time: [21.19], best model: 1
Epoch: 2, train_loss: 0.439

/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_32257/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))


Model Structure:  GRUD(
  (zl): Linear(in_features=285, out_features=77, bias=True)
  (rl): Linear(in_features=285, out_features=77, bias=True)
  (hl): Linear(in_features=285, out_features=77, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=77, bias=True)
  (fc): Linear(in_features=77, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.6990488, valid_loss: 0.6827437, time: [26.28], best model: 1
Epoch: 1, train_loss: 0.47661573, valid_loss: 0.47362897, time: [26.], best model: 1
Epoch: 2, train_loss: 0.38248751, valid_loss: 0.37671873, time: [26.34], best model: 1
Epoch: 3, train_loss: 0.3434369, valid_loss: 0.34586273, time: [25.51], best model: 1
Epoch: 4, train_loss: 0.32775455, valid_loss: 0.33284345, time: [25.35]

...

Model Structure:  GRUD(
  (zl): Linear(in_features=293, out_features=85, bias=True)
  (rl): Linear(in_features=293, out_features=85, bias=True)
  (hl): Linear(in_features=293, out_features=85, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=85, bias=True)
  (fc): Linear(in_features=85, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.73624838, valid_loss: 0.73817958, time: [18.89], best model: 1
Epoch: 1, train_loss: 0.54394829, valid_loss: 0.54507592, time: [18.68], best model: 1
Epoch: 2, train_loss: 0.43096374, valid_loss: 0.44112504, time: [18.42], best model: 1
Epoch: 3, train_loss: 0.37917816, valid_loss: 0.3894475, time: [18.53], best model: 1
Epoch: 4, train_loss: 0.34419742, valid_loss: 0.35989991, time: [18

/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_32257/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))


Model Structure:  GRUD(
  (zl): Linear(in_features=278, out_features=70, bias=True)
  (rl): Linear(in_features=278, out_features=70, bias=True)
  (hl): Linear(in_features=278, out_features=70, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=70, bias=True)
  (fc): Linear(in_features=70, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.71916224, valid_loss: 0.68640319, time: [18.16], best model: 1
Epoch: 1, train_loss: 0.50770448, valid_loss: 0.49813459, time: [18.19], best model: 1
Epoch: 2, train_loss: 0.40904296, valid_loss: 0.40834187, time: [18.37], best model: 1
Epoch: 3, train_loss: 0.36377756, valid_loss: 0.37074934, time: [18.2], best model: 1
Epoch: 4, train_loss: 0.33678249, valid_loss: 0.34629491, time: [18

/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_32257/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))


Model Structure:  GRUD(
  (zl): Linear(in_features=291, out_features=83, bias=True)
  (rl): Linear(in_features=291, out_features=83, bias=True)
  (hl): Linear(in_features=291, out_features=83, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=83, bias=True)
  (fc): Linear(in_features=83, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.73041529, valid_loss: 0.70824454, time: [21.39], best model: 1
Epoch: 1, train_loss: 0.52391106, valid_loss: 0.51166518, time: [21.36], best model: 1
Epoch: 2, train_loss: 0.41463323, valid_loss: 0.4127146, time: [21.55], best model: 1
Epoch: 3, train_loss: 0.36959414, valid_loss: 0.36132718, time: [21.54], best model: 1
Epoch: 4, train_loss: 0.34232188, valid_loss: 0.34345205, time: [21

In [21]:
y_score = np.concatenate(probabilities_test)[:, 1]
y_pred  = np.concatenate(probabilities_test).argmax(axis=1)
y_true  = np.concatenate(labels_test)

y_score  = np.nan_to_num(y_score , nan=0, posinf=0)
y_pred  = np.nan_to_num(y_pred , nan=0, posinf=0)
y_true  = np.nan_to_num(y_true , nan=0, posinf=0)

auc   = roc_auc_score(y_true, y_score)
auprc = average_precision_score(y_true, y_score)
acc   = accuracy_score(y_true, y_pred)
F1    = f1_score(y_true, y_pred)
print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
print(auc, auprc, acc, F1)

results[model_name][t][n] = None, best_hyperparams, auc, auprc, acc, F1, best_s
#with open('../scratch/mmd/baselines_gru-d.pkl', mode='wb') as f: pickle.dump(results, f)

Final results for model GRU-D on target mort_icu with representation lvl2
0.8693046451999644 0.4606874919945089 0.937368642286675 0.4398496240601504


In [22]:
%%time
model_name       = 'GRU-D'
hyperparams_list = GRU_D_hyperparams_list
RERUN            = False
if model_name not in results: results[model_name] = {}
for t in [
#    'mort_icu',
    'los_3', 
#    'mort_hosp', 
#    'los_7'
]:
    if t not in results[model_name]: results[model_name][t] = {}
    for n, X_train, X_dev, X_test in (
        ('lvl2', lvl2_train, lvl2_dev, lvl2_test),
#         ('raw', raw_train, raw_dev, raw_test)
    ):
        print("Running model %s on target %s with representation %s" % (model_name, t, n))
        X_mean = np.nanmean(
            to_3D_tensor(
                X_train.loc[:, pd.IndexSlice[:, 'mean']] * 
                np.where((X_train.loc[:, pd.IndexSlice[:, 'mask']] == 1).values, 1, np.NaN)
            ),
            axis=0, keepdims=True
        ).transpose([0, 2, 1])
        base_params = {'X_mean': X_mean, 'output_last': True, 'input_size': X_mean.shape[2]}
    
        if n in results[model_name][t]:
            if not RERUN: 
                print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
                print(results[model_name][t][n])
                continue
            best_s, best_hyperparams = results[model_name][t][n][-1], results[model_name][t][n][1]
            print("Loading best hyperparams", best_hyperparams)
        else:
            best_s, best_hyperparams = -np.Inf, None
            for i, hyperparams in enumerate(hyperparams_list):
                print("On sample %d / %d (hyperparams = %s)" % (i+1, len(hyperparams_list), repr((hyperparams))))

                early_stop_frac,batch_size,seed = [hyperparams[k] for k in ('early_stop_frac','batch_size','seed')]
                batch_size = int(batch_size)

                np.random.seed(seed)
                all_train_subjects = list(
                    np.random.permutation(Ys_train.index.get_level_values('subject_id').values)
                )
                N_early_stop        = int(len(all_train_subjects) * early_stop_frac)
                train_subjects      = all_train_subjects[:-N_early_stop]
                early_stop_subjects = all_train_subjects[-N_early_stop:]
                X_train_obs         = X_train[X_train.index.get_level_values('subject_id').isin(train_subjects)]
                Ys_train_obs        = Ys_train[Ys_train.index.get_level_values('subject_id').isin(train_subjects)]

                X_train_early_stop  = X_train[X_train.index.get_level_values('subject_id').isin(early_stop_subjects)]
                Ys_train_early_stop = Ys_train[
                    Ys_train.index.get_level_values('subject_id').isin(early_stop_subjects)
                ]

                train_dataloader      = prepare_dataloader(X_train_obs, Ys_train_obs[t], batch_size=batch_size)
                early_stop_dataloader = prepare_dataloader(
                    X_train_early_stop, Ys_train_early_stop[t], batch_size=batch_size
                )
                dev_dataloader        = prepare_dataloader(X_dev, Ys_dev[t], batch_size=batch_size)
                test_dataloader       = prepare_dataloader(X_test, Ys_test[t], batch_size=batch_size)

                model_hyperparams = copy.copy(base_params)
                model_hyperparams.update(
                    {k: v for k, v in hyperparams.items() if k in ('cell_size', 'hidden_size', 'batch_size')}
                )
                
                batch_size = int(batch_size)
                model = GRUD(**model_hyperparams)

                best_model, _ = Train_Model(
                    model, train_dataloader, early_stop_dataloader,
                    **{k: v for k, v in hyperparams.items() if k in (
                        'num_epochs', 'patience', 'learning_rate', 'batch_size'
                    )}
                )

                probabilities_dev, labels_dev = predict_proba(best_model, dev_dataloader)
                probabilities_dev = np.concatenate(probabilities_dev)[:, 1]
                labels_dev        = np.concatenate(labels_dev)
                s = roc_auc_score(labels_dev, probabilities_dev)
                if s > best_s:
                    best_s, best_hyperparams = s, hyperparams
                    print("New Best Score: %.2f @ hyperparams = %s" % (100*best_s, repr((best_hyperparams))))
                
        ## Test
        try:
            np.random.seed(seed)
            hyperparams = best_hyperparams # In case I forgot a replace below
            early_stop_frac,batch_size,seed = [best_hyperparams[k] for k in ('early_stop_frac','batch_size','seed')]
            batch_size = int(batch_size)
            X_train_concat, Ys_train_concat = pd.concat((X_train, X_dev)), pd.concat((Ys_train, Ys_dev))

            all_train_subjects = list(np.random.permutation(Ys_train_concat.index.get_level_values('subject_id').values))
            N_early_stop = int(len(all_train_subjects) * early_stop_frac)
            train_subjects, early_stop_subjects = all_train_subjects[:-N_early_stop], all_train_subjects[-N_early_stop:]
            X_train_obs         = X_train_concat[X_train_concat.index.get_level_values('subject_id').isin(train_subjects)]
            Ys_train_obs        = Ys_train_concat[Ys_train_concat.index.get_level_values('subject_id').isin(train_subjects)]

            X_train_early_stop  = X_train_concat[X_train_concat.index.get_level_values('subject_id').isin(early_stop_subjects)]
            Ys_train_early_stop = Ys_train_concat[Ys_train_concat.index.get_level_values('subject_id').isin(early_stop_subjects)]

            train_dataloader      = prepare_dataloader(X_train_obs, Ys_train_obs[t], batch_size=batch_size)
            early_stop_dataloader = prepare_dataloader(X_train_early_stop, Ys_train_early_stop[t], batch_size=batch_size)
            test_dataloader       = prepare_dataloader(X_test, Ys_test[t], batch_size=batch_size)

            model_hyperparams = copy.copy(base_params)
            model_hyperparams.update(
                {k: v for k, v in best_hyperparams.items() if k in ('cell_size', 'hidden_size', 'batch_size')}
            )
            model = GRUD(**model_hyperparams)

            best_model, (losses_train, losses_early_stop, losses_epochs_train, losses_epochs_early_stop) = Train_Model(
                model, train_dataloader, early_stop_dataloader,
                **{k: v for k, v in best_hyperparams.items() if k in (
                    'num_epochs', 'patience', 'learning_rate', 'batch_size'
                )}
            )

            probabilities_test, labels_test = predict_proba(best_model, test_dataloader)

            y_score = np.concatenate(probabilities_test)[:, 1]
            y_pred  = np.argmax(probabilities_test)
            y_true  = np.concatenate(labels_test)

            auc   = roc_auc_score(y_true, y_score)
            auprc = average_precision_score(y_true, y_score)
            acc   = accuracy_score(y_true, y_pred)
            F1    = f1_score(y_true, y_pred)
            print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
            print(auc, auprc, acc, F1)

            results[model_name][t][n] = None, best_hyperparams, auc, auprc, acc, F1, best_s
            #with open('../scratch/mmd/baselines_gru-d.pkl', mode='wb') as f: pickle.dump(results, f)
        except: pass

Running model GRU-D on target los_3 with representation lvl2


/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_32257/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))
<timed exec>:17: RuntimeWarning: Mean of empty slice


On sample 1 / 10 (hyperparams = {'cell_size': 55, 'hidden_size': 66, 'learning_rate': 0.07052195003967596, 'num_epochs': 22, 'patience': 6, 'batch_size': 60, 'early_stop_frac': 0.051828827734419186, 'seed': 9496})
Model Structure:  GRUD(
  (zl): Linear(in_features=274, out_features=66, bias=True)
  (rl): Linear(in_features=274, out_features=66, bias=True)
  (hl): Linear(in_features=274, out_features=66, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=66, bias=True)
  (fc): Linear(in_features=66, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.77214079, valid_loss: 0.79046267, time: [17.8], best model: 1
Epoch: 1, train_loss: 0.68401695, valid_loss: 0.72066018, time: [17.94], best model: 1
Epoch: 2, train_loss: 0.648

/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_32257/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))


Model Structure:  GRUD(
  (zl): Linear(in_features=285, out_features=77, bias=True)
  (rl): Linear(in_features=285, out_features=77, bias=True)
  (hl): Linear(in_features=285, out_features=77, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=77, bias=True)
  (fc): Linear(in_features=77, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.75962762, valid_loss: 0.76301143, time: [20.96], best model: 1
Epoch: 1, train_loss: 0.6661809, valid_loss: 0.683951, time: [20.89], best model: 1
Epoch: 2, train_loss: 0.63917993, valid_loss: 0.65185642, time: [20.97], best model: 1
Epoch: 3, train_loss: 0.6211725, valid_loss: 0.63763483, time: [21.04], best model: 1
Epoch: 4, train_loss: 0.62542174, valid_loss: 0.6378561, time: [20.97]

...

Model Structure:  GRUD(
  (zl): Linear(in_features=293, out_features=85, bias=True)
  (rl): Linear(in_features=293, out_features=85, bias=True)
  (hl): Linear(in_features=293, out_features=85, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=85, bias=True)
  (fc): Linear(in_features=85, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.77436729, valid_loss: 0.78037101, time: [19.25], best model: 1
Epoch: 1, train_loss: 0.69544001, valid_loss: 0.70304993, time: [18.58], best model: 1
Epoch: 2, train_loss: 0.65290113, valid_loss: 0.66264244, time: [18.56], best model: 1
Epoch: 3, train_loss: 0.6342501, valid_loss: 0.6411583, time: [18.59], best model: 1
Epoch: 4, train_loss: 0.62557931, valid_loss: 0.64158227, time: [18.

/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_32257/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))


Model Structure:  GRUD(
  (zl): Linear(in_features=278, out_features=70, bias=True)
  (rl): Linear(in_features=278, out_features=70, bias=True)
  (hl): Linear(in_features=278, out_features=70, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=70, bias=True)
  (fc): Linear(in_features=70, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.775225, valid_loss: 0.78779061, time: [18.3], best model: 1
Epoch: 1, train_loss: 0.68345955, valid_loss: 0.72459669, time: [18.47], best model: 1
Epoch: 2, train_loss: 0.64362507, valid_loss: 0.67705162, time: [18.26], best model: 1
Epoch: 3, train_loss: 0.63153915, valid_loss: 0.65354172, time: [18.3], best model: 1
Epoch: 4, train_loss: 0.62110492, valid_loss: 0.64944478, time: [18.27

/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_32257/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))


Model Structure:  GRUD(
  (zl): Linear(in_features=301, out_features=93, bias=True)
  (rl): Linear(in_features=301, out_features=93, bias=True)
  (hl): Linear(in_features=301, out_features=93, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=93, bias=True)
  (fc): Linear(in_features=93, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.76283926, valid_loss: 0.7740104, time: [24.74], best model: 1
Epoch: 1, train_loss: 0.6718209, valid_loss: 0.6984997, time: [24.4], best model: 1
Epoch: 2, train_loss: 0.6363845, valid_loss: 0.66689208, time: [24.39], best model: 1
Epoch: 3, train_loss: 0.62646111, valid_loss: 0.65657543, time: [24.42], best model: 1
Epoch: 4, train_loss: 0.61740643, valid_loss: 0.64855138, time: [24.58]

In [23]:
y_score = np.concatenate(probabilities_test)[:, 1]
y_pred  = np.concatenate(probabilities_test).argmax(axis=1)
y_true  = np.concatenate(labels_test)

y_score  = np.nan_to_num(y_score , nan=0, posinf=0)
y_pred  = np.nan_to_num(y_pred , nan=0, posinf=0)
y_true  = np.nan_to_num(y_true , nan=0, posinf=0)

auc   = roc_auc_score(y_true, y_score)
auprc = average_precision_score(y_true, y_score)
acc   = accuracy_score(y_true, y_pred)
F1    = f1_score(y_true, y_pred)
print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
print(auc, auprc, acc, F1)

#results[model_name][t][n] = None, best_hyperparams, auc, auprc, acc, F1, best_s
with open('../scratch/mmd/baselines_gru-d.pkl', mode='wb') as f: pickle.dump(results, f)

Final results for model GRU-D on target los_3 with representation lvl2
0.7291738052814327 0.6728083576052762 0.6927257525083612 0.601409978308026


In [24]:
%%time
model_name       = 'GRU-D'
hyperparams_list = GRU_D_hyperparams_list
RERUN            = False
if model_name not in results: results[model_name] = {}
for t in [
#    'mort_icu',
#    'los_3', 
    'mort_hosp', 
#    'los_7'
]:
    if t not in results[model_name]: results[model_name][t] = {}
    for n, X_train, X_dev, X_test in (
        ('lvl2', lvl2_train, lvl2_dev, lvl2_test),
#         ('raw', raw_train, raw_dev, raw_test)
    ):
        print("Running model %s on target %s with representation %s" % (model_name, t, n))
        X_mean = np.nanmean(
            to_3D_tensor(
                X_train.loc[:, pd.IndexSlice[:, 'mean']] * 
                np.where((X_train.loc[:, pd.IndexSlice[:, 'mask']] == 1).values, 1, np.NaN)
            ),
            axis=0, keepdims=True
        ).transpose([0, 2, 1])
        base_params = {'X_mean': X_mean, 'output_last': True, 'input_size': X_mean.shape[2]}
    
        if n in results[model_name][t]:
            if not RERUN: 
                print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
                print(results[model_name][t][n])
                continue
            best_s, best_hyperparams = results[model_name][t][n][-1], results[model_name][t][n][1]
            print("Loading best hyperparams", best_hyperparams)
        else:
            best_s, best_hyperparams = -np.Inf, None
            for i, hyperparams in enumerate(hyperparams_list):
                print("On sample %d / %d (hyperparams = %s)" % (i+1, len(hyperparams_list), repr((hyperparams))))

                early_stop_frac,batch_size,seed = [hyperparams[k] for k in ('early_stop_frac','batch_size','seed')]
                batch_size = int(batch_size)

                np.random.seed(seed)
                all_train_subjects = list(
                    np.random.permutation(Ys_train.index.get_level_values('subject_id').values)
                )
                N_early_stop        = int(len(all_train_subjects) * early_stop_frac)
                train_subjects      = all_train_subjects[:-N_early_stop]
                early_stop_subjects = all_train_subjects[-N_early_stop:]
                X_train_obs         = X_train[X_train.index.get_level_values('subject_id').isin(train_subjects)]
                Ys_train_obs        = Ys_train[Ys_train.index.get_level_values('subject_id').isin(train_subjects)]

                X_train_early_stop  = X_train[X_train.index.get_level_values('subject_id').isin(early_stop_subjects)]
                Ys_train_early_stop = Ys_train[
                    Ys_train.index.get_level_values('subject_id').isin(early_stop_subjects)
                ]

                train_dataloader      = prepare_dataloader(X_train_obs, Ys_train_obs[t], batch_size=batch_size)
                early_stop_dataloader = prepare_dataloader(
                    X_train_early_stop, Ys_train_early_stop[t], batch_size=batch_size
                )
                dev_dataloader        = prepare_dataloader(X_dev, Ys_dev[t], batch_size=batch_size)
                test_dataloader       = prepare_dataloader(X_test, Ys_test[t], batch_size=batch_size)

                model_hyperparams = copy.copy(base_params)
                model_hyperparams.update(
                    {k: v for k, v in hyperparams.items() if k in ('cell_size', 'hidden_size', 'batch_size')}
                )
                
                batch_size = int(batch_size)
                model = GRUD(**model_hyperparams)

                best_model, _ = Train_Model(
                    model, train_dataloader, early_stop_dataloader,
                    **{k: v for k, v in hyperparams.items() if k in (
                        'num_epochs', 'patience', 'learning_rate', 'batch_size'
                    )}
                )

                probabilities_dev, labels_dev = predict_proba(best_model, dev_dataloader)
                probabilities_dev = np.concatenate(probabilities_dev)[:, 1]
                labels_dev        = np.concatenate(labels_dev)
                s = roc_auc_score(labels_dev, probabilities_dev)
                if s > best_s:
                    best_s, best_hyperparams = s, hyperparams
                    print("New Best Score: %.2f @ hyperparams = %s" % (100*best_s, repr((best_hyperparams))))
                
        ## Test
        try:
            np.random.seed(seed)
            hyperparams = best_hyperparams # In case I forgot a replace below
            early_stop_frac,batch_size,seed = [best_hyperparams[k] for k in ('early_stop_frac','batch_size','seed')]
            batch_size = int(batch_size)
            X_train_concat, Ys_train_concat = pd.concat((X_train, X_dev)), pd.concat((Ys_train, Ys_dev))

            all_train_subjects = list(np.random.permutation(Ys_train_concat.index.get_level_values('subject_id').values))
            N_early_stop = int(len(all_train_subjects) * early_stop_frac)
            train_subjects, early_stop_subjects = all_train_subjects[:-N_early_stop], all_train_subjects[-N_early_stop:]
            X_train_obs         = X_train_concat[X_train_concat.index.get_level_values('subject_id').isin(train_subjects)]
            Ys_train_obs        = Ys_train_concat[Ys_train_concat.index.get_level_values('subject_id').isin(train_subjects)]

            X_train_early_stop  = X_train_concat[X_train_concat.index.get_level_values('subject_id').isin(early_stop_subjects)]
            Ys_train_early_stop = Ys_train_concat[Ys_train_concat.index.get_level_values('subject_id').isin(early_stop_subjects)]

            train_dataloader      = prepare_dataloader(X_train_obs, Ys_train_obs[t], batch_size=batch_size)
            early_stop_dataloader = prepare_dataloader(X_train_early_stop, Ys_train_early_stop[t], batch_size=batch_size)
            test_dataloader       = prepare_dataloader(X_test, Ys_test[t], batch_size=batch_size)

            model_hyperparams = copy.copy(base_params)
            model_hyperparams.update(
                {k: v for k, v in best_hyperparams.items() if k in ('cell_size', 'hidden_size', 'batch_size')}
            )
            model = GRUD(**model_hyperparams)

            best_model, (losses_train, losses_early_stop, losses_epochs_train, losses_epochs_early_stop) = Train_Model(
                model, train_dataloader, early_stop_dataloader,
                **{k: v for k, v in best_hyperparams.items() if k in (
                    'num_epochs', 'patience', 'learning_rate', 'batch_size'
                )}
            )

            probabilities_test, labels_test = predict_proba(best_model, test_dataloader)

            y_score = np.concatenate(probabilities_test)[:, 1]
            y_pred  = np.argmax(probabilities_test)
            y_true  = np.concatenate(labels_test)

            auc   = roc_auc_score(y_true, y_score)
            auprc = average_precision_score(y_true, y_score)
            acc   = accuracy_score(y_true, y_pred)
            F1    = f1_score(y_true, y_pred)
            print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
            print(auc, auprc, acc, F1)

            results[model_name][t][n] = None, best_hyperparams, auc, auprc, acc, F1, best_s
            #with open('../scratch/mmd/baselines_gru-d.pkl', mode='wb') as f: pickle.dump(results, f)
        except: pass

Running model GRU-D on target mort_hosp with representation lvl2


/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_32257/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))
<timed exec>:17: RuntimeWarning: Mean of empty slice


On sample 1 / 10 (hyperparams = {'cell_size': 55, 'hidden_size': 66, 'learning_rate': 0.07052195003967596, 'num_epochs': 22, 'patience': 6, 'batch_size': 60, 'early_stop_frac': 0.051828827734419186, 'seed': 9496})
Model Structure:  GRUD(
  (zl): Linear(in_features=274, out_features=66, bias=True)
  (rl): Linear(in_features=274, out_features=66, bias=True)
  (hl): Linear(in_features=274, out_features=66, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=66, bias=True)
  (fc): Linear(in_features=66, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.71195475, valid_loss: 0.73059521, time: [17.09], best model: 1
Epoch: 1, train_loss: 0.5293986, valid_loss: 0.54717411, time: [16.97], best model: 1
Epoch: 2, train_loss: 0.448

/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_32257/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))


Model Structure:  GRUD(
  (zl): Linear(in_features=285, out_features=77, bias=True)
  (rl): Linear(in_features=285, out_features=77, bias=True)
  (hl): Linear(in_features=285, out_features=77, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=77, bias=True)
  (fc): Linear(in_features=77, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.6810011, valid_loss: 0.68089394, time: [20.91], best model: 1
Epoch: 1, train_loss: 0.48470182, valid_loss: 0.48294277, time: [20.8], best model: 1
Epoch: 2, train_loss: 0.41155875, valid_loss: 0.41593893, time: [20.85], best model: 1
Epoch: 3, train_loss: 0.38114501, valid_loss: 0.39024298, time: [20.88], best model: 1
Epoch: 4, train_loss: 0.369368, valid_loss: 0.37732786, time: [20.89

...

Model Structure:  GRUD(
  (zl): Linear(in_features=293, out_features=85, bias=True)
  (rl): Linear(in_features=293, out_features=85, bias=True)
  (hl): Linear(in_features=293, out_features=85, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=85, bias=True)
  (fc): Linear(in_features=85, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.72280925, valid_loss: 0.73065571, time: [18.75], best model: 1
Epoch: 1, train_loss: 0.53858983, valid_loss: 0.53064164, time: [18.7], best model: 1
Epoch: 2, train_loss: 0.44868279, valid_loss: 0.45509277, time: [18.76], best model: 1
Epoch: 3, train_loss: 0.4123719, valid_loss: 0.41663136, time: [18.77], best model: 1
Epoch: 4, train_loss: 0.38257356, valid_loss: 0.3936047, time: [18.7

/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_32257/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))


Model Structure:  GRUD(
  (zl): Linear(in_features=278, out_features=70, bias=True)
  (rl): Linear(in_features=278, out_features=70, bias=True)
  (hl): Linear(in_features=278, out_features=70, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=70, bias=True)
  (fc): Linear(in_features=70, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.7058971, valid_loss: 0.68836616, time: [18.12], best model: 1
Epoch: 1, train_loss: 0.51538627, valid_loss: 0.49926025, time: [18.09], best model: 1
Epoch: 2, train_loss: 0.43171611, valid_loss: 0.43008999, time: [18.21], best model: 1
Epoch: 3, train_loss: 0.39353092, valid_loss: 0.40784879, time: [18.19], best model: 1
Epoch: 4, train_loss: 0.36908848, valid_loss: 0.38669268, time: [18

/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_32257/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))


Model Structure:  GRUD(
  (zl): Linear(in_features=293, out_features=85, bias=True)
  (rl): Linear(in_features=293, out_features=85, bias=True)
  (hl): Linear(in_features=293, out_features=85, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=85, bias=True)
  (fc): Linear(in_features=85, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.7080739, valid_loss: 0.70019276, time: [21.34], best model: 1
Epoch: 1, train_loss: 0.51753883, valid_loss: 0.49879297, time: [21.31], best model: 1
Epoch: 2, train_loss: 0.43283826, valid_loss: 0.43343672, time: [21.36], best model: 1
Epoch: 3, train_loss: 0.39175201, valid_loss: 0.39963297, time: [21.38], best model: 1
Epoch: 4, train_loss: 0.37723235, valid_loss: 0.38361272, time: [21

In [25]:
y_score = np.concatenate(probabilities_test)[:, 1]
y_pred  = np.concatenate(probabilities_test).argmax(axis=1)
y_true  = np.concatenate(labels_test)

y_score  = np.nan_to_num(y_score , nan=0, posinf=0)
y_pred  = np.nan_to_num(y_pred , nan=0, posinf=0)
y_true  = np.nan_to_num(y_true , nan=0, posinf=0)

auc   = roc_auc_score(y_true, y_score)
auprc = average_precision_score(y_true, y_score)
acc   = accuracy_score(y_true, y_pred)
F1    = f1_score(y_true, y_pred)
print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
print(auc, auprc, acc, F1)

results[model_name][t][n] = None, best_hyperparams, auc, auprc, acc, F1, best_s
#with open('../scratch/mmd/baselines_gru-d.pkl', mode='wb') as f: pickle.dump(results, f)

Final results for model GRU-D on target mort_hosp with representation lvl2
0.8591123691751954 0.5051294711456881 0.9107628004179729 0.45326504481434055


In [26]:
%%time
model_name       = 'GRU-D'
hyperparams_list = GRU_D_hyperparams_list
RERUN            = False
if model_name not in results: results[model_name] = {}
for t in [
#    'mort_icu',
#    'los_3', 
#    'mort_hosp', 
     'los_7'
]:
    if t not in results[model_name]: results[model_name][t] = {}
    for n, X_train, X_dev, X_test in (
        ('lvl2', lvl2_train, lvl2_dev, lvl2_test),
#         ('raw', raw_train, raw_dev, raw_test)
    ):
        print("Running model %s on target %s with representation %s" % (model_name, t, n))
        X_mean = np.nanmean(
            to_3D_tensor(
                X_train.loc[:, pd.IndexSlice[:, 'mean']] * 
                np.where((X_train.loc[:, pd.IndexSlice[:, 'mask']] == 1).values, 1, np.NaN)
            ),
            axis=0, keepdims=True
        ).transpose([0, 2, 1])
        base_params = {'X_mean': X_mean, 'output_last': True, 'input_size': X_mean.shape[2]}
    
        if n in results[model_name][t]:
            if not RERUN: 
                print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
                print(results[model_name][t][n])
                continue
            best_s, best_hyperparams = results[model_name][t][n][-1], results[model_name][t][n][1]
            print("Loading best hyperparams", best_hyperparams)
        else:
            best_s, best_hyperparams = -np.Inf, None
            for i, hyperparams in enumerate(hyperparams_list):
                print("On sample %d / %d (hyperparams = %s)" % (i+1, len(hyperparams_list), repr((hyperparams))))

                early_stop_frac,batch_size,seed = [hyperparams[k] for k in ('early_stop_frac','batch_size','seed')]
                batch_size = int(batch_size)

                np.random.seed(seed)
                all_train_subjects = list(
                    np.random.permutation(Ys_train.index.get_level_values('subject_id').values)
                )
                N_early_stop        = int(len(all_train_subjects) * early_stop_frac)
                train_subjects      = all_train_subjects[:-N_early_stop]
                early_stop_subjects = all_train_subjects[-N_early_stop:]
                X_train_obs         = X_train[X_train.index.get_level_values('subject_id').isin(train_subjects)]
                Ys_train_obs        = Ys_train[Ys_train.index.get_level_values('subject_id').isin(train_subjects)]

                X_train_early_stop  = X_train[X_train.index.get_level_values('subject_id').isin(early_stop_subjects)]
                Ys_train_early_stop = Ys_train[
                    Ys_train.index.get_level_values('subject_id').isin(early_stop_subjects)
                ]

                train_dataloader      = prepare_dataloader(X_train_obs, Ys_train_obs[t], batch_size=batch_size)
                early_stop_dataloader = prepare_dataloader(
                    X_train_early_stop, Ys_train_early_stop[t], batch_size=batch_size
                )
                dev_dataloader        = prepare_dataloader(X_dev, Ys_dev[t], batch_size=batch_size)
                test_dataloader       = prepare_dataloader(X_test, Ys_test[t], batch_size=batch_size)

                model_hyperparams = copy.copy(base_params)
                model_hyperparams.update(
                    {k: v for k, v in hyperparams.items() if k in ('cell_size', 'hidden_size', 'batch_size')}
                )
                
                batch_size = int(batch_size)
                model = GRUD(**model_hyperparams)

                best_model, _ = Train_Model(
                    model, train_dataloader, early_stop_dataloader,
                    **{k: v for k, v in hyperparams.items() if k in (
                        'num_epochs', 'patience', 'learning_rate', 'batch_size'
                    )}
                )

                probabilities_dev, labels_dev = predict_proba(best_model, dev_dataloader)
                probabilities_dev = np.concatenate(probabilities_dev)[:, 1]
                labels_dev        = np.concatenate(labels_dev)
                s = roc_auc_score(labels_dev, probabilities_dev)
                if s > best_s:
                    best_s, best_hyperparams = s, hyperparams
                    print("New Best Score: %.2f @ hyperparams = %s" % (100*best_s, repr((best_hyperparams))))
                
        ## Test
        try:
            np.random.seed(seed)
            hyperparams = best_hyperparams # In case I forgot a replace below
            early_stop_frac,batch_size,seed = [best_hyperparams[k] for k in ('early_stop_frac','batch_size','seed')]
            batch_size = int(batch_size)
            X_train_concat, Ys_train_concat = pd.concat((X_train, X_dev)), pd.concat((Ys_train, Ys_dev))

            all_train_subjects = list(np.random.permutation(Ys_train_concat.index.get_level_values('subject_id').values))
            N_early_stop = int(len(all_train_subjects) * early_stop_frac)
            train_subjects, early_stop_subjects = all_train_subjects[:-N_early_stop], all_train_subjects[-N_early_stop:]
            X_train_obs         = X_train_concat[X_train_concat.index.get_level_values('subject_id').isin(train_subjects)]
            Ys_train_obs        = Ys_train_concat[Ys_train_concat.index.get_level_values('subject_id').isin(train_subjects)]

            X_train_early_stop  = X_train_concat[X_train_concat.index.get_level_values('subject_id').isin(early_stop_subjects)]
            Ys_train_early_stop = Ys_train_concat[Ys_train_concat.index.get_level_values('subject_id').isin(early_stop_subjects)]

            train_dataloader      = prepare_dataloader(X_train_obs, Ys_train_obs[t], batch_size=batch_size)
            early_stop_dataloader = prepare_dataloader(X_train_early_stop, Ys_train_early_stop[t], batch_size=batch_size)
            test_dataloader       = prepare_dataloader(X_test, Ys_test[t], batch_size=batch_size)

            model_hyperparams = copy.copy(base_params)
            model_hyperparams.update(
                {k: v for k, v in best_hyperparams.items() if k in ('cell_size', 'hidden_size', 'batch_size')}
            )
            model = GRUD(**model_hyperparams)

            best_model, (losses_train, losses_early_stop, losses_epochs_train, losses_epochs_early_stop) = Train_Model(
                model, train_dataloader, early_stop_dataloader,
                **{k: v for k, v in best_hyperparams.items() if k in (
                    'num_epochs', 'patience', 'learning_rate', 'batch_size'
                )}
            )

            probabilities_test, labels_test = predict_proba(best_model, test_dataloader)

            y_score = np.concatenate(probabilities_test)[:, 1]
            y_pred  = np.argmax(probabilities_test)
            y_true  = np.concatenate(labels_test)

            auc   = roc_auc_score(y_true, y_score)
            auprc = average_precision_score(y_true, y_score)
            acc   = accuracy_score(y_true, y_pred)
            F1    = f1_score(y_true, y_pred)
            print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
            print(auc, auprc, acc, F1)

            results[model_name][t][n] = None, best_hyperparams, auc, auprc, acc, F1, best_s
            #with open('../scratch/mmd/baselines_gru-d.pkl', mode='wb') as f: pickle.dump(results, f)
        except: pass

Running model GRU-D on target los_7 with representation lvl2


/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_32257/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))
<timed exec>:17: RuntimeWarning: Mean of empty slice


On sample 1 / 10 (hyperparams = {'cell_size': 55, 'hidden_size': 66, 'learning_rate': 0.07052195003967596, 'num_epochs': 22, 'patience': 6, 'batch_size': 60, 'early_stop_frac': 0.051828827734419186, 'seed': 9496})
Model Structure:  GRUD(
  (zl): Linear(in_features=274, out_features=66, bias=True)
  (rl): Linear(in_features=274, out_features=66, bias=True)
  (hl): Linear(in_features=274, out_features=66, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=66, bias=True)
  (fc): Linear(in_features=66, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.76649232, valid_loss: 0.76589677, time: [17.36], best model: 1
Epoch: 1, train_loss: 0.56668247, valid_loss: 0.56333681, time: [17.3], best model: 1
Epoch: 2, train_loss: 0.475

/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_32257/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))


Model Structure:  GRUD(
  (zl): Linear(in_features=285, out_features=77, bias=True)
  (rl): Linear(in_features=285, out_features=77, bias=True)
  (hl): Linear(in_features=285, out_features=77, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=77, bias=True)
  (fc): Linear(in_features=77, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.71176369, valid_loss: 0.70877583, time: [21.3], best model: 1
Epoch: 1, train_loss: 0.51091309, valid_loss: 0.50683903, time: [21.23], best model: 1
Epoch: 2, train_loss: 0.42681627, valid_loss: 0.43145986, time: [21.28], best model: 1
Epoch: 3, train_loss: 0.38457507, valid_loss: 0.38830693, time: [21.22], best model: 1
Epoch: 4, train_loss: 0.37822036, valid_loss: 0.37400307, time: [21

...

Model Structure:  GRUD(
  (zl): Linear(in_features=293, out_features=85, bias=True)
  (rl): Linear(in_features=293, out_features=85, bias=True)
  (hl): Linear(in_features=293, out_features=85, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=85, bias=True)
  (fc): Linear(in_features=85, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.74817345, valid_loss: 0.74840753, time: [18.78], best model: 1
Epoch: 1, train_loss: 0.56560163, valid_loss: 0.57626103, time: [18.54], best model: 1
Epoch: 2, train_loss: 0.48450172, valid_loss: 0.48844892, time: [18.51], best model: 1
Epoch: 3, train_loss: 0.4327135, valid_loss: 0.42678874, time: [18.6], best model: 1
Epoch: 4, train_loss: 0.39233916, valid_loss: 0.40081714, time: [18.

/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_32257/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))


Model Structure:  GRUD(
  (zl): Linear(in_features=278, out_features=70, bias=True)
  (rl): Linear(in_features=278, out_features=70, bias=True)
  (hl): Linear(in_features=278, out_features=70, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=70, bias=True)
  (fc): Linear(in_features=70, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.7329578, valid_loss: 0.75003911, time: [18.22], best model: 1
Epoch: 1, train_loss: 0.5464761, valid_loss: 0.54030503, time: [18.18], best model: 1
Epoch: 2, train_loss: 0.46216321, valid_loss: 0.44928696, time: [30.47], best model: 1
Epoch: 3, train_loss: 0.40639872, valid_loss: 0.41128947, time: [18.31], best model: 1
Epoch: 4, train_loss: 0.38002378, valid_loss: 0.37891992, time: [18.

/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_32257/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))


Model Structure:  GRUD(
  (zl): Linear(in_features=301, out_features=93, bias=True)
  (rl): Linear(in_features=301, out_features=93, bias=True)
  (hl): Linear(in_features=301, out_features=93, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=93, bias=True)
  (fc): Linear(in_features=93, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.71920952, valid_loss: 0.72912818, time: [24.35], best model: 1
Epoch: 1, train_loss: 0.51797197, valid_loss: 0.52304631, time: [24.53], best model: 1
Epoch: 2, train_loss: 0.44228131, valid_loss: 0.4447233, time: [28.78], best model: 1
Epoch: 3, train_loss: 0.39819701, valid_loss: 0.40179992, time: [24.56], best model: 1
Epoch: 4, train_loss: 0.37994169, valid_loss: 0.3849639, time: [24.

In [27]:
y_score = np.concatenate(probabilities_test)[:, 1]
y_pred  = np.concatenate(probabilities_test).argmax(axis=1)
y_true  = np.concatenate(labels_test)

y_score  = np.nan_to_num(y_score , nan=0, posinf=0)
y_pred  = np.nan_to_num(y_pred , nan=0, posinf=0)
y_true  = np.nan_to_num(y_true , nan=0, posinf=0)

auc   = roc_auc_score(y_true, y_score)
auprc = average_precision_score(y_true, y_score)
acc   = accuracy_score(y_true, y_pred)
F1    = f1_score(y_true, y_pred)
print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
print(auc, auprc, acc, F1)

results[model_name][t][n] = None, best_hyperparams, auc, auprc, acc, F1, best_s
with open('../scratch/mmd/baselines_gru-d.pkl', mode='wb') as f: pickle.dump(results, f)

Final results for model GRU-D on target los_7 with representation lvl2
0.7307566320119379 0.1720343822007 0.9138795986622074 0.07207207207207207
